<a href="https://colab.research.google.com/github/sribarrow/audiorecording/blob/master/cnn_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import keras
from keras import backend as K
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import LSTM, ConvLSTM2D, TimeDistributed, Conv2D, MaxPooling2D, Dropout
from keras.layers import Dense, Activation, Flatten, Reshape
from keras import optimizers, regularizers

Using TensorFlow backend.


Reference: https://colab.research.google.com/notebooks/gpu.ipynb#scrollTo=3IEVK-KFxi5Z

In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
from tensorflow.python.client import device_lib

In [5]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2264822376879559134, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4650713512067990364
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5074167125349027635
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14912199066
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1323827267544679216
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [6]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
# with open('/gdrive/My Drive/foo.txt', 'w') as f:
#   f.write('Hello Google Drive!')
# !cat '/gdrive/My Drive/foo.txt'

Reference: https://github.com/tensorflow/tensorflow/issues/17076

Fix for ERROR - report_tensor_allocations_upon_oom

In [0]:
# config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 1} ) 
#  sess = tf.Session(config=config) 
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [9]:
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[22. 28.]
 [49. 64.]]


In [10]:
from tensorflow.core.protobuf import config_pb2
sess.run(c, options=config_pb2.RunOptions(
        report_tensor_allocations_upon_oom=True))

array([[22., 28.],
       [49., 64.]], dtype=float32)

Reference: https://stackoverflow.com/questions/46066850/understanding-the-resourceexhaustederror-oom-when-allocating-tensor-with-shape

Fix for ERROR - ResourceExhaustedError: OOM when allocating tensor with shape

In [0]:
gpu_options = tf.GPUOptions(allow_growth=True)

In [0]:
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [0]:
train_path = '/gdrive/My Drive/raag_images/train'
valid_path = '/gdrive/My Drive/raag_images/check'
test_path = '/gdrive/My Drive/raag_images/test'

In [14]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(300,300), classes=['mohanam','kamboji','kalyani'],  batch_size=10)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path,  target_size=(300,300), classes=['mohanam','kamboji','kalyani'],  batch_size=6)
test_batches = ImageDataGenerator().flow_from_directory(test_path,  target_size=(300,300), classes=['mohanam','kamboji','kalyani'],  batch_size=10)

Found 160 images belonging to 3 classes.
Found 52 images belonging to 3 classes.
Found 0 images belonging to 3 classes.


In [15]:
gpu_options = tf.GPUOptions(allow_growth=True)
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
model = Sequential()
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu'), input_shape=(5, 300, 300, 3)))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(1, 1))))

model.add(TimeDistributed(Conv2D(128, (4,4), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Flatten()))
model.add(Dropout(0.5))

model.add(LSTM(256, return_sequences=False, dropout=0.5))

model.add(Dense(3, activation='softmax'))

model.compile(optimizers.Adam(lr=0.0001),loss="categorical_crossentropy",metrics=["accuracy"])



In [40]:
model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_49 (TimeDis (None, 5, 298, 298, 64)   1792      
_________________________________________________________________
time_distributed_50 (TimeDis (None, 5, 297, 297, 64)   0         
_________________________________________________________________
time_distributed_51 (TimeDis (None, 5, 294, 294, 128)  131200    
_________________________________________________________________
time_distributed_52 (TimeDis (None, 5, 147, 147, 128)  0         
_________________________________________________________________
time_distributed_53 (TimeDis (None, 5, 2765952)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 5, 2765952)        0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 256)             

In [0]:
#model.add(Conv2D(32, (3, 3),input_shape=(300,300,3)))
#keras.layers.ConvLSTM2D(32, (3, 3), strides=(1, 1))
#model.add(TimeDistributed(Dense(32)))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Flatten())

#model.add(Dense(64, activation='relu'))
#model.add(Dense(32, activation='relu'))
#model.add(Dense(3, activation='softmax'))
#model.compile(optimizers.Adam(lr=0.0001),loss="categorical_crossentropy",metrics=["accuracy"])


NameError: ignored

In [0]:
#model.add(TimeDistributed(Conv2D(32, (3, 3),input_shape=(300,300,3))))
#model.add(TimeDistributed(ConvLSTM2D(32, (3, 3), strides=(1, 1),  activation='sigmoid')))
#model.add(TimeDistributed(Conv2D(64, (3, 3), strides=(1,1), activation='sigmoid')))
#model.add(TimeDistributed(ConvLSTM2D(64, (3, 3), strides=(1, 1),  activation='sigmoid')))
#model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
#model.add(TimeDistributed(Conv2D(128, (3, 3), strides=(1,1), activation='sigmoid')))
#model.add(TimeDistributed(ConvLSTM2D(128, (3, 3), strides=(1, 1),  activation='sigmoid')))
#model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
#model.add(Dropout(0.2))
#model.add(Dense(128, activation='sigmoid'))
#model.add(TimeDistributed(Flatten()))
#model.add(Dense(3, activation='softmax'))
#model.add(Reshape((-1, 300, 300,1)))
#model.compile(optimizers.Adam(lr=0.0001),loss="categorical_crossentropy",metrics=["accuracy"])


In [0]:

#
#model.compile(optimizers.rmsprop(lr=0.0001),loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

ValueError: ignored

In [34]:
STEP_SIZE_TRAIN=train_batches.n//train_batches.batch_size
print(STEP_SIZE_TRAIN)
STEP_SIZE_VALID=valid_batches.n//valid_batches.batch_size
print(STEP_SIZE_VALID)


16
8


In [43]:
model.fit((5, 300, 300, 3), epochs=10)

AttributeError: ignored

In [41]:
model.fit_generator(train_batches, steps_per_epoch=STEP_SIZE_TRAIN, epochs=50, validation_data=valid_batches, validation_steps=STEP_SIZE_VALID)

Epoch 1/50


ValueError: ignored

In [0]:
model.evaluate_generator(generator=valid_batches, steps=STEP_SIZE_VALID)

[8.730635017156601, 0.45833333395421505]

**Fine tuning the model using pretrained model VGG16**

In [0]:
from keras.layers import Input

In [0]:
from keras.layers.pooling import AveragePooling2D

In [0]:
from keras.applications.vgg16 import VGG16

In [0]:
#vgg16_model = keras.applications.vgg16.VGG16()
#vgg16_model.layers.pop()

In [0]:
vgg16_model = keras.applications.vgg16.VGG16(include_top=False, input_tensor=Input(shape=(300, 300, 3)))

58892288/58889256 [==============================] - 3s 0us/step


In [0]:
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300, 300, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 75, 128)       0     

Fine tuning Reference: https://www.pyimagesearch.com/2019/06/24/change-input-shape-dimensions-for-fine-tuning-with-keras/

**Print the type of this vgg16 model.**

*Note this is of type model and not sequential*


In [0]:
type(vgg16_model)

keras.engine.training.Model

In [0]:
new_model = vgg16_model.output

In [0]:
new_model = AveragePooling2D(pool_size=(4, 4))(new_model)

In [0]:
new_model = Flatten(name="flatten")(new_model)
new_model = Dense(128, activation="relu")(new_model)
new_model = Dropout(0.5)(new_model)
new_model = Dense(3, activation="softmax")(new_model)

In [0]:
from keras.models import Model
model = Model(inputs=vgg16_model.input, outputs=new_model)

In [0]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300, 300, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 75, 128)       0   

**Transform the above model to sequential**

In [0]:
for l in vgg16_model.layers:
  layer_trainable=False

In [0]:
seq_model = Sequential()
type(seq_model)
print(seq_model)

**Add layers from vgg16 to our model**

In [0]:
for l in model.layers:
  print(l)
  seq_model.add(l)

**Remove output layer and change it to fit previous**

In [0]:
seq_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 75, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 75, 75, 256)      

In [0]:
opt = optimizers.Adam(lr=0.001)

seq_model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [0]:
seq_model.fit_generator(train_batches, steps_per_epoch=STEP_SIZE_TRAIN, epochs=5, validation_data=valid_batches, validation_steps=STEP_SIZE_VALID, verbose=2)

Epoch 1/5
 - 20s - loss: 8.0789 - acc: 0.4938 - val_loss: 8.7306 - val_acc: 0.4583
Epoch 2/5
 - 14s - loss: 8.0590 - acc: 0.5000 - val_loss: 7.7087 - val_acc: 0.5217
Epoch 3/5
 - 13s - loss: 7.9583 - acc: 0.5063 - val_loss: 9.8110 - val_acc: 0.3913
Epoch 4/5
 - 13s - loss: 8.1598 - acc: 0.4938 - val_loss: 9.4606 - val_acc: 0.4130
Epoch 5/5
 - 13s - loss: 7.9583 - acc: 0.5063 - val_loss: 8.0590 - val_acc: 0.5000


In [0]:
seq_model.evaluate_generator(generator=valid_batches, steps=STEP_SIZE_VALID)

[9.402222156524658, 0.4166666716337204]

In [0]:
session.close()